In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
%%capture
!pip install libauc==1.2.0
!pip install medmnist
!pip install torchio

In [3]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

from libauc.models import resnet18
from libauc.sampler import DualSampler
from libauc.metrics import auc_prc_score

import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image

from torchio import Image

from libauc.losses import AUCMLoss, CrossEntropyLoss
from libauc.optimizers import PESG, Adam
from libauc.models import densenet121 as DenseNet121


from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torchvision.models as models
# from torch.optim import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.2.1 @ https://github.com/MedMNIST/MedMNIST/


In [5]:
from torchio.transforms import RandomAffine, RandomFlip, RandomNoise, RandomGamma


class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=28, crop_size=24, mode='train'):
        self.images = images
        self.targets = targets
        self.mode = mode
        
        self.transform_test = transforms.Compose([RandomAffine(p=0)])
        
        
        # for loss function
        self.pos_indices = np.flatnonzero(targets==1)
        self.pos_index_map = {}
        for i, idx in enumerate(self.pos_indices):
            self.pos_index_map[idx] = i

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
#         image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
           idx = self.pos_index_map[idx] if idx in self.pos_indices else -1
           image = self.transform_train(image)
        else:
           image = self.transform_test(image)
        return idx, image, target 

In [6]:
device = torch.device(0 if torch.cuda.is_available() else 'cpu')

In [10]:
def getTestAUC(model_PATH, model, model_fc):
    print(f'Testing ...')  
    PATH = model_PATH

    num_classes = 1

    model.fc = model_fc


    model = model.to(device)

    model_state_dict = torch.load(PATH)
    model.load_state_dict(model_state_dict)  # Load the saved parameters into the model
    with torch.no_grad():    
         test_pred = []
         test_true = []
         for jdx, (index, data, labels) in enumerate(test_loader):
             test_data, test_labels = data, labels
             test_data=test_data.repeat(1,3,1,1,1)
             test_labels=test_labels.float()
             test_data = test_data.to(device)
             y_pred = model(test_data.float())
             test_pred.append(y_pred.cpu().detach().numpy())
             test_true.append(test_labels.numpy())

         test_true = np.concatenate(test_true)
         test_pred = np.concatenate(test_pred)
         val_auc_mean =  roc_auc_score(test_true, test_pred) 
         print('Val_AUC=%.4f'%( val_auc_mean))

### NoduleMNIST3D

In [19]:
data_flag = 'nodulemnist3d'
#data_flag = 'adrenalmnist3d'
# data_flag = 'vesselmnist3d'
# data_flag = 'synapsemnist3d'

download = True

info = INFO[data_flag]

DataClass = getattr(medmnist, info['python_class'])
test_dataset = DataClass(split='test',  download=download)
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
batch_size = 32
testSet = ImageDataset(test_images, test_labels, mode='test')
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=True, num_workers=2)

num_classes = 1
model = models.video.r3d_18(pretrained=False)
input_channel = model.fc.in_features
model_fc = nn.Sequential(nn.Linear(input_channel, 128),
                         nn.ReLU(),
#                                  nn.Dropout(p=0.2),
                         nn.Linear(128, 32),
                         nn.ReLU(),
#                                  nn.Dropout(p=0.1),
                         nn.Linear(32, num_classes),
                         nn.Sigmoid()
                         )

model_PATH = "/kaggle/input/medmnist3d-models/nodule3d_resnet3d_pesg_0.1_SLR_RLRP_noDP_9303.pth"
getTestAUC(model_PATH, model, model_fc)

Using downloaded and verified file: /root/.medmnist/nodulemnist3d.npz


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Testing ...
Val_AUC=0.9263


### AdrenalMNIST3D

In [21]:
# data_flag = 'nodulemnist3d'
data_flag = 'adrenalmnist3d'
# data_flag = 'vesselmnist3d'
# data_flag = 'synapsemnist3d'

download = True

info = INFO[data_flag]

DataClass = getattr(medmnist, info['python_class'])
test_dataset = DataClass(split='test',  download=download)
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
batch_size = 32
testSet = ImageDataset(test_images, test_labels, mode='test')
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=True, num_workers=2)

num_classes = 1
model = models.video.r3d_18(pretrained=False)
input_channel = model.fc.in_features
model_fc = nn.Sequential(nn.Linear(input_channel, 128),
                         nn.ReLU(),
#                                  nn.Dropout(p=0.2),
                         nn.Linear(128, 32),
                         nn.ReLU(),
#                                  nn.Dropout(p=0.1),
                         nn.Linear(32, num_classes),
                         nn.Sigmoid()
                         )

model_PATH = "/kaggle/input/medmnist3d-models/adrenal3d_resnet3d_pesg_0.1_SLR_RLRP_noDP_8720.pth"
getTestAUC(model_PATH, model, model_fc)

Using downloaded and verified file: /root/.medmnist/adrenalmnist3d.npz


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Testing ...
Val_AUC=0.8676


### SynapseMNIST3D

In [39]:
# data_flag = 'nodulemnist3d'
# data_flag = 'adrenalmnist3d'
# data_flag = 'vesselmnist3d'
data_flag = 'synapsemnist3d'

download = True

info = INFO[data_flag]

DataClass = getattr(medmnist, info['python_class'])
test_dataset = DataClass(split='test',  download=download)
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
batch_size = 16
testSet = ImageDataset(test_images, test_labels, mode='test')
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=True, num_workers=2)

num_classes = 1
model = models.video.r3d_18(pretrained=False)
input_channel = model.fc.in_features
model_fc = nn.Sequential(nn.Linear(input_channel, 128),
                         nn.ReLU(),
#                          nn.Dropout(p=0.2),
                         nn.Linear(128, 32),
                         nn.ReLU(),
#                          nn.Dropout(p=0.1),
                         nn.Linear(32, num_classes),
                         nn.Sigmoid()
                         )

model_PATH = "/kaggle/input/medmnist3d-models/synapse3d_resnet3d_pesg_0.05_SLR_RLRP_16batchsize_8632.pth"
getTestAUC(model_PATH, model, model_fc)

100%|██████████| 38034583/38034583 [01:10<00:00, 538082.42it/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Testing ...
Val_AUC=0.8623


### VesselMNIST3D

In [46]:
# data_flag = 'nodulemnist3d'
# data_flag = 'adrenalmnist3d'
data_flag = 'vesselmnist3d'
# data_flag = 'synapsemnist3d'

download = True

info = INFO[data_flag]

DataClass = getattr(medmnist, info['python_class'])
test_dataset = DataClass(split='test',  download=download)
test_images = np.array([np.asarray(image) for (image, target) in test_dataset])
test_labels = [target for (image, target) in test_dataset]
batch_size = 32
testSet = ImageDataset(test_images, test_labels, mode='test')
test_loader = torch.utils.data.DataLoader(testSet, batch_size=batch_size, shuffle=True, num_workers=2)

num_classes = 1
model = models.video.r3d_18(pretrained=False)
input_channel = model.fc.in_features
model_fc = nn.Sequential(nn.Linear(input_channel, 128),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(128, 32),
                         nn.ReLU(),
                         nn.Dropout(p=0.1),
                         nn.Linear(32, num_classes),
                         nn.Sigmoid()
                         )

model_PATH = "/kaggle/input/medmnist3d-vessel/vessel3d_resnet3d_pesg_0.1_SLR_RLRP_9696.pth"
getTestAUC(model_PATH, model, model_fc)

Using downloaded and verified file: /root/.medmnist/vesselmnist3d.npz


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Testing ...
Val_AUC=0.9652
